# Booking.com Feature Extraction
---

Download the file BookingDotCom_HotelReviews.xlsx from Canvas. This file contains over 515,000 guest reviews and rating of almost 1500 hotels across Europe scraped from popular hotel reservation website Booking.com. The text data was cleaned by removing unicode and punctuation and transformed to lower case. No other preprocessing was done. More information on each field is provided in the "Data Description" tab of the Excel file.

        1. What are the top five hotel features (e.g., location, staff, etc.) that customers mention the most in positive reviews and top five features they mention most in negative reviews? Your identified features must make sense (e.g., "great" or "negative" are not features). (3 points)
        
        2. What are the top five features that customers prefer most if they are a solo traveler vs traveling with a group vs on a business trip vs a leisure trip vs traveling as a couple vs a family with young children. You will find these categories in the "Tags" column. There are a few more tags that we don't need. (2 points).

        3. What are the top five features customers like most and top five features they complain about most about hotels in United Kingdom, France, Italy, and Spain? Country information is available inside Hotel_Address. (2 points)
        
        4. Create a dashboard with the following plots; (1) "Top Five Hotels Overall" with consistently high ratings, (2) Bottom Five Hotels Overall" with consistently low ratings, (3) Five Most Improved Hotels" with the highest improvement in average ratings from 2015 to 2017, showing their average ratings for each of the three years. (0.5+0.5+2 points).

Write clear, compact, and understandable code with comment/markdown statements as appropriate. Non-working code or unnecessary code will be penalized. 

Submit your Jupyter file using the link below or provide a link to your Google Colab or Github file.


In [45]:
# import packages to use
import pandas as pd
import re
import pycountry
import ast
import spacy
from sklearn.feature_extraction.text  import CountVectorizer

In [25]:
# load dataframe
df = pd.read_excel("BookingDotCom_HotelReviews.xlsx", sheet_name="Data")

# sample the first 1000 rows of df
df = df[:1000]

# rename df columns to lower case
df.columns= df.columns.str.lower()

'\n\n'

In [26]:
# remove numbers from text using re
df['positive_comments'] = df['positive_comments'].apply(lambda x: re.sub(r'\d+', '', x))
df['negative_comments'] = df['negative_comments'].apply(lambda x: re.sub(r'\d+', '', x))

In [75]:
# lemmatize, remove stopwords and convert to lower the columns: positive_comments and negative_comments

load_model = spacy.load('en_core_web_sm', disable = ['parser','ner'])

# remove stopwords
myStopwords = list(load_model.Defaults.stop_words)
myStopwords.append(['negative','positive', 'stay', 'hotel', 'night', 'book', 'com'])

def lemmaFunc(text):
    doc = load_model(text)
    allowed_tags = ['NOUN', 'PROPN']        # to get only nouns and proper nouns
    return " ".join([token.lemma_.lower() for token in doc if token.pos_ in allowed_tags \
                          and token.text not in myStopwords
                          ])

df['positive_comments'] = df['positive_comments'].apply(lambda x: lemmaFunc(x))
df['negative_comments'] = df['negative_comments'].apply(lambda x: lemmaFunc(x))

In [39]:
'''
Create column with country name; we get the list of countries from the pycountry package
and use the hotel_address column to extract country name
'''

df['country'] = df["hotel_address"].apply(
    lambda address: ' '.join([c.name for c in pycountry.countries if c.name in address])
    )

# use the review_date column to extract the year and store in new column
df['year'] = pd.DatetimeIndex(df['review_date']).year

In [42]:
'''
In this step we deal with the tags column using the steps defined below:
    1. Define tags we are interested in
    2. Define a function to apply to tags column to remove tags we are not interested in by:
            - Converting the individual row values to list (from string) e.g. "[' Leisure trip ']" -> [' leisure trip ']
            - Strip the whitespaces from individual elements e.g. [' leisure trip ']-> ['leisure trip']
            - Drop tags we are not interested in
'''

# customer tags we are interested in
customer_tags = ['solo traveler','group','business trip','leisure trip','couple','family with young children']


def clean_tag(x):

    # convert value from string to a list
    myTags = ast.literal_eval(x.lower())

    # strip whitespaces from elements and drop those we are not interested in
    myTags = [customerTag.strip() for customerTag in myTags if customerTag.strip() in customer_tags]

    return myTags


# apply clean_tag() function to the tags column
df['tags'] = df['tags'].apply(lambda x: clean_tag(x))


'''
Function applied to tags column to extract new columns for customer categories
Lambda function will be used as in the steps below
'''

def split_tag(x:list, tagName:str) -> int:
    
    t = [1 if tagName in x else 0][0]
    
    return t

# dictionary of column names (new additional columns) and customer tags (as contained in tags of interest)
tagDict = {
    'solo_traveler' : 'solo traveler',
    'group' : 'group',
    'business_trip' : 'business trip',
    'leisure_trip' : 'leisure trip',
    'couple' : 'couple',
    'family_with_young_children' : 'family with young children'
}

# applying function on tags column to get new separated columns
for key, value in tagDict.items():
    df[key] = df['tags'].apply(lambda x: split_tag(x, value))

In [96]:
'''
Getting word frequencies using CountVectorizer from sklearn
'''

vecPos = CountVectorizer(
                        strip_accents='ascii', stop_words='english', ngram_range=(1,1),
                        analyzer='word', max_df=0.95,max_features=50
                        )

vecNeg = CountVectorizer(max_df=0.85, ngram_range=(1,2), max_features=50)

sparseVecPos = vecPos.fit_transform(df['positive_comments'])   # You can fit and transform jointly 
sparseVecNeg = vecNeg.fit_transform(df['negative_comments'])

matPos = pd.DataFrame(sparseVecPos.toarray(), columns=vecPos.get_feature_names_out())
matNeg = pd.DataFrame(sparseVecNeg.toarray(), columns=vecNeg.get_feature_names_out())

In [97]:
# 1a. What are the top five hotel features (e.g., location, staff, etc.) that customers mention the most in positive reviews
matPos.agg(sum).sort_values(ascending=False)[:5]

room        61
location    39
park        38
staff       35
bed         29
dtype: int64

In [98]:
# 1b. What are the top five hotel features (e.g., location, staff, etc.) that customers mention the most in negative reviews
matNeg.agg(sum).sort_values(ascending=False)[:5]

room         116
bathroom      22
breakfast     20
floor         18
shower        16
dtype: int64

In [125]:
'''
What are the top five features that customers prefer most if they are a solo traveler vs traveling with a 
group vs on a business trip vs a leisure trip vs traveling as a couple vs a family with young children.
'''
# indices of customer group data subsets (by travel tag)
soloIndex = df.index[df.solo_traveler != 0] # solo_traveler
groupIndex = df.index[df.group != 0] # group
businessIndex = df.index[df.business_trip != 0] # business_trip
leisureIndex = df.index[df.leisure_trip != 0] # leisure_trip
coupleIndex = df.index[df.couple != 0] # couple
familyIndex = df.index[df.family_with_young_children != 0] # family_with_young_children


In [123]:
# solo
matPos.iloc[soloIndex].agg(sum).sort_values(ascending=False)[:5]

location     7
room         6
style        4
breakfast    4
park         3
dtype: int64

In [126]:
# group
matPos.iloc[groupIndex].agg(sum).sort_values(ascending=False)[:5]

room          12
park           5
building       5
restaurant     4
staff          4
dtype: int64

In [112]:
# business
matPos.iloc[businessIndex].agg(sum).sort_values(ascending=False)[:5]

room        5
park        4
style       4
location    4
building    3
dtype: int64

In [113]:
# leisure
matPos.iloc[leisureIndex].agg(sum).sort_values(ascending=False)[:5]

room        55
staff       34
location    34
park        33
bed         28
dtype: int64

In [114]:
# couple
matPos.iloc[coupleIndex].agg(sum).sort_values(ascending=False)[:5]

room        40
park        27
staff       24
location    24
bed         23
dtype: int64

In [115]:
# family with young children
matPos.iloc[familyIndex].agg(sum).sort_values(ascending=False)[:5]

location     6
staff        5
room         3
park         3
breakfast    2
dtype: int64

In [ ]:
'''
What are the top five features customers like most and top five features they complain about most about hotels 
in United Kingdom, France, Italy, and Spain? Country information is available inside Hotel_Address. (2 points)
'''

In [147]:
# United Kingdom likes/complaints
print(matPos.iloc[df.index[df.country == 'United Kingdom']].agg(sum).sort_values(ascending=False)[:5])
print('\n')
print(matNeg.iloc[df.index[df.country == 'United Kingdom']].agg(sum).sort_values(ascending=False)[:5])

access     0.0
room       0.0
morning    0.0
museum     0.0
park       0.0
dtype: float64


area         0.0
sound        0.0
place        0.0
price        0.0
reception    0.0
dtype: float64


In [148]:
# France likes/complaints
print(matPos.iloc[df.index[df.country == 'France']].agg(sum).sort_values(ascending=False)[:5])
print('\n')
print(matNeg.iloc[df.index[df.country == 'France']].agg(sum).sort_values(ascending=False)[:5])

access     0.0
room       0.0
morning    0.0
museum     0.0
park       0.0
dtype: float64


area         0.0
sound        0.0
place        0.0
price        0.0
reception    0.0
dtype: float64


In [149]:
# Italy likes/complaints
print(matPos.iloc[df.index[df.country == 'Italy']].agg(sum).sort_values(ascending=False)[:5])
print('\n')
print(matNeg.iloc[df.index[df.country == 'Italy']].agg(sum).sort_values(ascending=False)[:5])

access     0.0
room       0.0
morning    0.0
museum     0.0
park       0.0
dtype: float64


area         0.0
sound        0.0
place        0.0
price        0.0
reception    0.0
dtype: float64


In [150]:
# Spain likes/complaints
print(matPos.iloc[df.index[df.country == 'Spain']].agg(sum).sort_values(ascending=False)[:5])
print('\n')
print(matPos.iloc[df.index[df.country == 'Spain']].agg(sum).sort_values(ascending=False)[:5])

access     0.0
room       0.0
morning    0.0
museum     0.0
park       0.0
dtype: float64


access     0.0
room       0.0
morning    0.0
museum     0.0
park       0.0
dtype: float64
